In [27]:
# pip install tensorflow

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import mne

In [2]:
folder = r'D:\CHB_MIT'
# folder = r'E:\New folder\chb24'
file_path = [os.path.join(folder,i ) for i in os.listdir(folder)]

In [3]:
print(file_path)

['D:\\CHB_MIT\\chb24']


In [4]:
os.listdir(file_path[0])

['ictal', 'interictal', 'postictal', 'preictal']

In [ ]:
import os
import mne

# folder = r'E:\New folder\chb24'
subfolders = [os.path.join(folder, name) for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

# Dictionary to store loaded data
data_dict = {}

for subfolder in subfolders:
    for state_folder in os.listdir(subfolder):
        full_state_path = os.path.join(subfolder, state_folder)
        fif_files = [f for f in os.listdir(full_state_path) if f.endswith('.fif')]
        for fif_file in fif_files:
            file_path = os.path.join(full_state_path, fif_file)
            raw = mne.io.read_raw_fif(file_path, preload=True)
            data_dict[(subfolder, state_folder, fif_file)] = raw


# raw data process 24

In [72]:
import numpy as np
import mne

def process_eeg_data(data_dict, target_sfreq=128, epoch_duration=5.0):
    """
    Processes EEG data by downsampling, creating epochs with overlap, 
    and converting data to reduced precision format.
    
    Parameters:
    - data_dict: Dictionary with (subject, state, file_name) as keys and raw EEG data as values.
    - target_sfreq: Target sampling frequency for downsampling.
    - epoch_duration: Duration of each epoch in seconds.
    
    Returns:
    - X: Combined NumPy array of EEG epochs.
    - Y: Corresponding labels for the epochs.
    """
    
    X = []
    Y = []
    
    for (subject, state, file_name), raw in data_dict.items():
        raw.resample(target_sfreq)  # Downsample the data
        
        if raw.times[-1] < epoch_duration:
            print(f"Warning: Data length is shorter than the epoch duration for {file_name}.")
            continue

        # Determine overlap and label based on state
        if state == 'ictal':
            overlap_fraction = 0.985  # 95% overlap
            label = 1
        elif state == 'preictal':
            overlap_fraction = 0.50  # 50% overlap
            label = 2
        elif state == 'postictal':  # interictal, postictal
            overlap_fraction = 0.80  # 90% overlap
            label = 3
        else:
            overlap_fraction = 0.0  # No overlap
            label = 0

        overlap = epoch_duration * overlap_fraction
        
        # Create fixed-length epochs with the calculated overlap
        epochs = mne.make_fixed_length_epochs(raw, duration=epoch_duration, preload=True, overlap=overlap)
        
        # Get data and labels, using reduced precision
        X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True
        Y_epochs = np.full(X_epochs.shape[0], label)  # Assign the label to each epoch
        print(f"Label {label} generated {len(Y_epochs)} epochs.")
        
        X.append(X_epochs)
        Y.append(Y_epochs)

    # Combine all data and labels into single arrays using vstack and hstack
    X = np.vstack(X)
    Y = np.hstack(Y)

    print("Final shapes:", X.shape, Y.shape)
    
    return X, Y

# Example usage
# X, Y = process_eeg_data(data_dict)


In [73]:
X, Y = process_eeg_data(data_dict,target_sfreq=128, epoch_duration=5.0)

Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
267 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 267 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 267 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
267 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 267 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 267 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
321 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 321 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 321 epochs.
Sampling frequency of the instance is already 128.0, returning unmo

C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True
C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True
C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True
C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will 

Label 1 generated 361 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
294 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 294 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 294 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
187 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 187 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 187 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
254 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 254 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 254 epochs.
Sampling frequency of the instance is

C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True
C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True
C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True
C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will 

Not setting metadata
187 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 187 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 187 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
867 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 867 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 867 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
147 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 147 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 147 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
294 matching events found
No baseline correcti

...

0 bad epochs dropped
Label 2 generated 719 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
719 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 719 events and 640 original time points ...


C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True


0 bad epochs dropped
Label 2 generated 719 epochs.


C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True


Final shapes: (25062, 21, 640) (25062,)


In [74]:
pd.DataFrame(Y).value_counts()
# # Y

0
0    6693
2    6319
3    6294
1    5756
Name: count, dtype: int64

In [75]:
X.shape

(25062, 21, 640)

In [76]:
X = np.moveaxis(X,1,2)
X.shape

(25062, 640, 21)

In [77]:
np.savez(r'D:\processed data for CHB\chb24\raw\5sraw.npz',X=X, Y=Y)

# DWT data process different time segments


In [78]:
import numpy as np
import mne
import pywt  # Ensure PyWavelets is installed

def process_eeg_data_with_dwt(data_dict, target_sfreq=128, epoch_duration, wavelet='db4', dwt_level=5):
    """
    Processes EEG data by downsampling, creating epochs with overlap,
    applying DWT to each epoch, and converting data to reduced precision format.
    
    Parameters:
    - data_dict: Dictionary with (subject, state, file_name) as keys and raw EEG data as values.
    - target_sfreq: Target sampling frequency for downsampling.
    - epoch_duration: Duration of each epoch in seconds.
    - wavelet: Wavelet type to use for DWT.
    - dwt_level: Decomposition level for DWT.
    
    Returns:
    - X: Combined NumPy array of DWT-transformed EEG epochs.
    - Y: Corresponding labels for the epochs.
    """
    
    X = []
    Y = []
    
    for (subject, state, file_name), raw in data_dict.items():
        raw.resample(target_sfreq)  # Downsample the data
        
        if raw.times[-1] < epoch_duration:
            print(f"Warning: Data length is shorter than the epoch duration for {file_name}.")
            continue

        # Determine overlap and label based on state
        if state == 'ictal':
            overlap_fraction = 0.985  # 95% overlap
            label = 1
        elif state == 'preictal':
            overlap_fraction = 0.50  # 50% overlap
            label = 2
        elif state == 'postictal':  # interictal, postictal
            overlap_fraction = 0.80  # 80% overlap
            label = 3
        else:
            overlap_fraction = 0.0  # No overlap
            label = 0

        overlap = epoch_duration * overlap_fraction
        
        # Create fixed-length epochs with the calculated overlap
        epochs = mne.make_fixed_length_epochs(raw, duration=epoch_duration, preload=True, overlap=overlap)
        
        # Get data from the epochs
        X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Reduced precision
        
        # Apply DWT to each epoch and append to X
        for epoch in X_epochs:
            dwt_coeffs = [pywt.wavedec(channel, wavelet, level=dwt_level) for channel in epoch]
            X.append(dwt_coeffs)
        
        # Assign labels
        Y_epochs = np.full(len(X_epochs), label)  # Assign the label to each epoch
        Y.append(Y_epochs)

    # Convert to numpy arrays
    X = np.array(X)
    Y = np.hstack(Y)

    print("Final shapes:", X.shape, Y.shape)
    
    return X, Y


In [79]:
X, Y = process_eeg_data(data_dict,target_sfreq=128, epoch_duration=5.0)

Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
267 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 267 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 267 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
267 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 267 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 267 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
321 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 321 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 321 epochs.
Sampling frequency of the instance is already 128.0, returning unmo

C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True
C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True
C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True
C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will 

0 bad epochs dropped
Label 1 generated 361 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
294 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 294 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 294 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
187 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 187 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 187 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
254 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 254 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 254 epochs.
Sampling frequen

C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True
C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True
C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True
C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will 

Label 1 generated 227 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
187 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 187 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 187 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
867 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 867 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 867 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
147 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 147 events and 640 original time points ...
0 bad epochs dropped
Label 1 generated 147 epochs.
Sampling frequency of the instance is

...

0 bad epochs dropped
Label 2 generated 719 epochs.
Sampling frequency of the instance is already 128.0, returning unmodified.
Not setting metadata
719 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 719 events and 640 original time points ...


C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True


0 bad epochs dropped
Label 2 generated 719 epochs.


C:\Users\ACER\AppData\Local\Temp\ipykernel_11988\1072328186.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_epochs = epochs.get_data().astype(np.float32, copy=True)  # Explicitly set copy=True


Final shapes: (25062, 21, 640) (25062,)


In [80]:
pd.DataFrame(Y).value_counts()



0
0    6693
2    6319
3    6294
1    5756
Name: count, dtype: int64

In [81]:
X.shape

(25062, 21, 640)

In [82]:
X = np.moveaxis(X,1,2)
X.shape


(25062, 640, 21)

In [ ]:
np.savez(r'D:\processed data for CHB\chb24\raw\5sraw.npz',X=X, Y=Y)

# Model Run and test

In [11]:
from sklearn.preprocessing import StandardScaler
# c. Scale the data
n_channels = X.shape[2]
scalers = [StandardScaler() for _ in range(n_channels)]

for ch in range(n_channels):
    X[:, :, ch] = scalers[ch].fit_transform(X[:, :, ch])

print("Data scaling completed.")

Data scaling completed.


In [12]:
X = np.expand_dims(X, axis=-1)
print(X.shape) 

(13143, 2560, 21, 1)


In [20]:
# mask = (Y == 0) | (Y == 2)
# mask

In [21]:
import os
import numpy as np
import pandas as pd
import mne
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GRU, Dense, Flatten, Attention, Input, Multiply, Add,Reshape
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
# Assuming X and Y are already defined and preprocessed up to X.shape = (10770, 2560, 21)

# 1. Data Preparation
# a. Filter for interictal (0) and preictal (2)
mask = (Y == 0) | (Y == 2)
X_filtered = X[mask]
Y_filtered = Y[mask]
Y_filtered = np.where(Y_filtered == 0, 0, 1)  # Relabel

print("Class distribution after filtering:")
print(pd.Series(Y_filtered).value_counts())

# b. Split the data
X_train, X_test, Y_train, Y_test = train_test_split(
    X_filtered, Y_filtered, test_size=0.2, random_state=42, stratify=Y_filtered
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

# # c. Scale the data
# n_channels = X_train.shape[2]
# scalers = [StandardScaler() for _ in range(n_channels)]

# for ch in range(n_channels):
#     X_train[:, :, ch] = scalers[ch].fit_transform(X_train[:, :, ch])
#     X_test[:, :, ch] = scalers[ch].transform(X_test[:, :, ch])

# print("Data scaling completed.")

# 2. Handling Class Imbalance
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(Y_train),
    y=Y_train
)
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}
print("Class weights:", class_weights_dict)

# 3. Building the Model
input_shape = X_train.shape[1:]  # (2560, 21)
num_classes = 1  # Binary classification

# model = Sequential([
#     Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
#     BatchNormalization(),
#     MaxPooling1D(pool_size=3),
#     # Dropout(0.3),

#     Conv1D(filters=64, kernel_size=3, activation='relu'),
#     BatchNormalization(),
#     MaxPooling1D(pool_size=3),
#     # Dropout(0.3),

#     # Conv1D(filters=128, kernel_size=10, activation='relu'),


#     Flatten(),
#     Dense(256, activation='relu'),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(num_classes, activation='sigmoid')
# ])

# model.compile(
#     optimizer='adam',
#     loss='binary_crossentropy',
#     metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
# )
# Input layer
# Input layer
input_layer = Input(input_shape)



l2_reg = 0.0001  # Regularization factor

# Conv2D and Pooling layers with L2 regularization
conv1 = Conv2D(filters=32, kernel_size=(3, 1), strides=(1, 1), activation='relu',
               kernel_regularizer=l2(l2_reg))(input_layer)
maxpool1 = MaxPooling2D(pool_size=(3, 1), strides=(1, 1))(conv1)

conv2 = Conv2D(filters=64, kernel_size=(3, 1), strides=(1, 1), activation='relu',
               kernel_regularizer=l2(l2_reg))(maxpool1)
maxpool2 = MaxPooling2D(pool_size=(3, 1), strides=(1, 1))(conv2)

# Reshape for GRU input
reshaped = Reshape((maxpool2.shape[1], -1))(maxpool2)

# GRU Layer with L2 regularization
gru = GRU(10, return_sequences=True, kernel_regularizer=l2(l2_reg))(reshaped)

# Attention Layer
attention = Attention()([gru, gru])

# Multiply GRU output by attention scores
context_vector = Multiply()([gru, attention])

# Add context vector and GRU output
output = Add()([context_vector, gru])

# Fully connected layers with L2 regularization
fc = Flatten()(output)
fc = Dense(128, activation='relu', kernel_regularizer=l2(l2_reg))(fc)
output_layer = Dense(2, activation='softmax', kernel_regularizer=l2(l2_reg))(fc)

# Create model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()
# Hyperparameters configuration

l2_reg = 0.0001

# Add L2 Regularization
for layer in model.layers:
    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer = tf.keras.regularizers.l2(l2_reg)

# 4. Training the Model
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train, Y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_test, Y_test),
    class_weight=class_weights_dict,
    callbacks=[early_stop],
    verbose=1
)

# 5. Evaluating the Model
test_loss, test_accuracy, test_auc = model.evaluate(X_test, Y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test AUC: {test_auc:.4f}")

Y_pred_prob = model.predict(X_test).ravel()
Y_pred = (Y_pred_prob >= 0.5).astype(int)

print("Classification Report:")
print(classification_report(Y_test, Y_pred, target_names=['Interictal', 'Preictal']))

conf_matrix = confusion_matrix(Y_test, Y_pred)
print("Confusion Matrix:")
print(conf_matrix)

plt.figure(figsize=(6,5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Interictal', 'Preictal'],
            yticklabels=['Interictal', 'Preictal'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

# Plot Training History
plt.figure(figsize=(14,5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Over Epochs')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Over Epochs')
plt.legend()

plt.show()


Class distribution after filtering:
0    3346
1    3151
Name: count, dtype: int64
Training samples: 5197
Testing samples: 1300
Class weights: {0: 0.9710388639760837, 1: 1.0307417691392304}


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)    │ (None, 2560, 21, 1)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_14 (Conv2D)            │ (None, 2558, 21, 32)      │             128 │ input_layer_7[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_14              │ (None, 2556, 21, 32)      │               0 │ conv2d_14[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_15 (Conv2D)            │ (None, 2554, 21, 64)      │           6,208 │ max_pooling2d_14[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_15              │ (None, 2552, 21, 64)      │               0 │ conv2d_15[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ reshape_5 (Reshape)           │ (None, 2552, 1344)        │               0 │ max_pooling2d_15[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gru_3 (GRU)                   │ (None, 2552, 10)          │          40,680 │ reshape_5[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attention_1 (Attention)       │ (None, 2552, 10)          │               0 │ gru_3[0][0], gru_3[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multiply_1 (Multiply)         │ (None, 2552, 10)          │               0 │ gru_3[0][0],               │
│                               │                           │                 │ attention_1[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_1 (Add)                   │ (None, 2552, 10)          │               0 │ multiply_1[0][0],          │
│                               │                           │                 │ gru_3[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_5 (Flatten)           │ (None, 25520)             │               0 │ add_1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 128)               │       3,266,688 │ flatten_5[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 2)                 │             258 │ dense_2[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 3,313,962 (12.64 MB)

 Trainable params: 3,313,962 (12.64 MB)

 Non-trainable params: 0 (0.00 B)

AttributeError: property 'kernel_regularizer' of 'GRU' object has no setter